In [1]:
f = open("../api_key.txt", "r")
api_key = f.readline()

In [2]:
import requests
import json
from os.path import exists
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

from data_manager import grab_daily_data, get_stock_data, preprocess_data_rnn

In [14]:
data = grab_daily_data('DIS', api_key)
df = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index')
df = df.drop(labels='5. volume', axis=1)
df = df.sort_index()
df = df.astype(float)
df['Target'] = df['4. close'].astype(float).shift(-1)
df = df.dropna()
scaler = MinMaxScaler()
scaler = scaler.fit(df)
arr = scaler.transform(df)
df = pd.DataFrame(arr, columns=['open', 'high', 'low', 'close', 'target'])
df.head(n=15)

,open,high,low,close,target
0,0.061754,0.063307,0.062995,0.063357,0.066334
1,0.063095,0.064631,0.066073,0.066334,0.069682
2,0.072860,0.070294,0.072614,0.069682,0.067662
3,0.074845,0.078869,0.068821,0.067662,0.056022
4,0.062773,0.060343,0.058157,0.056022,0.051717
5,0.054404,0.053726,0.054420,0.051717,0.052408
6,0.052365,0.053091,0.055079,0.052408,0.053737
7,0.052365,0.053409,0.055079,0.053737,0.056022
8,0.055047,0.054415,0.054749,0.056022,0.060009
9,0.057408,0.063307,0.058157,0.060009,0.064367


In [4]:
x, y = preprocess_data_rnn('IBM', api_key)
len(x)

5738

In [17]:
def get_compiled_model():
    lstm_input = keras.Input(shape=(50,4), name="lstm_input")
    inputs = layers.LSTM(128, name="lstm_layer")(lstm_input)
    inputs = layers.Dense(128, name="dense_layer_1")(inputs)
    inputs = layers.Dense(1, name="dense_layer_2")(inputs)
    output = layers.Activation('linear', name='output')(inputs)
    model = keras.Model(inputs=lstm_input, outputs=output)
    model.compile(optimizer='adam', loss='mse')
    return model

In [18]:
def train_model(symbol, model, api_key, epochs=30):
    x, y = preprocess_data_rnn(symbol, api_key, lookback=50)
    model.fit(np.asarray(x), np.asarray(y), epochs=epochs, batch_size=128, shuffle=True, validation_split=0.3)
    return model

In [21]:
def predict_and_save(symbol, api_key, model, save=True):
    x, y = preprocess_data_rnn(symbol, api_key, looback=50)
    pred = model.predict(np.asarray(x))
    plt.plot(np.reshape(y, (len(y))))
    plt.plot(np.reshape(pred, (len(pred))))
    plt.legend(["Actual Price", "Predicted Price"])
    plt.show()
    if save:
        my_dict = {str(i): str(pred[i][0]) for i in range(0, len(pred))}
        with open("../data/" + symbol + "_pred_rnn.json", "w") as output_file:
            json.dump(my_dict, output_file)

In [19]:
model = get_compiled_model()
train_model('IBM', model, api_key, epochs=25)

Epoch 1/25
32/32 [==============================] - 4s 73ms/step - loss: 0.0120 - val_loss: 0.0013
Epoch 2/25
32/32 [==============================] - 2s 62ms/step - loss: 6.0258e-04 - val_loss: 5.5202e-04
Epoch 3/25
32/32 [==============================] - 2s 58ms/step - loss: 3.6484e-04 - val_loss: 4.2349e-04
Epoch 4/25
32/32 [==============================] - 2s 61ms/step - loss: 3.5410e-04 - val_loss: 4.1203e-04
Epoch 5/25
32/32 [==============================] - 2s 59ms/step - loss: 3.4598e-04 - val_loss: 4.6035e-04
Epoch 6/25
32/32 [==============================] - 2s 65ms/step - loss: 3.4207e-04 - val_loss: 3.8803e-04
Epoch 7/25
32/32 [==============================] - 3s 81ms/step - loss: 3.3177e-04 - val_loss: 3.8665e-04
Epoch 8/25
32/32 [==============================] - 3s 80ms/step - loss: 3.1834e-04 - val_loss: 3.8360e-04
Epoch 9/25
32/32 [==============================] - 3s 80ms/step - loss: 3.2563e-04 - val_loss: 3.6482e-04
Epoch 10/25
32/32 [==========================

In [22]:
predict_and_save('DIS', api_key, model, save=True)

TypeError: preprocess_data_rnn() got an unexpected keyword argument 'looback'